In [9]:
import os
import re
import json
from groq import Groq
from dotenv import load_dotenv

load_dotenv()


MODEL ="llama3-70b-8192"
GROQ_CLIENT = Groq()
#"llama3-groq-70b-8192-tool-use-preview

In [13]:
def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})
        
get_current_weather(location="Madrid", unit="celsius")        

'{"temperature": 25, "unit": "celsius"}'

In [5]:
#system prompt
TOOL_SYSTEM_PROMPT = """ 
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. 
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug 
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>"""

In [6]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]
agent_chat_history = []

user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}

tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)

output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


In [10]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)
    
    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [11]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

In [14]:
result = get_current_weather(**parsed_output["arguments"])
result

'{"temperature": 25, "unit": "celsius"}'

In [15]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

In [16]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'

Tool Decorator

In [50]:
import json
import requests
from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.tool_pattern.tool_agent import ToolAgent

def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. 
    Each story contains the title, URL, score, author, and time of submission. The data is fetched 
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    
    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors
        
        # Get the top story IDs
        top_story_ids = response.json()[:top_n]
        
        top_stories = []
        
        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()
            
            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })
        
        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [51]:
json.loads(fetch_top_hacker_news_stories(top_n=5))

[{'title': 'ESP32 Undocumented Bluetooth Commands: Clearing the Air',
  'url': 'https://developer.espressif.com/blog/2025/03/esp32-bluetooth-clearing-the-air/'},
 {'title': 'A Supersolid Made Using Photons',
  'url': 'https://www.photonicsonline.com/doc/a-super-solid-made-using-pho-tons-0001'},
 {'title': 'Show HN: Seven39, a social media app that is only open for 3 hours every evening',
  'url': 'https://www.seven39.com'},
 {'title': 'Bending Spacetime in the Basement (1997)',
  'url': 'https://www.fourmilab.ch/gravitation/foobar/'},
 {'title': 'Software-Defined Radio for Engineers (2018) [pdf]',
  'url': 'https://www.analog.com/media/en/training-seminars/design-handbooks/Software-Defined-Radio-for-Engineers-2018/SDR4Engineers.pdf'}]

In [52]:
hn_tool = tool(fetch_top_hacker_news_stories)

In [53]:
hn_tool.name

'fetch_top_hacker_news_stories'

In [54]:
json.loads(hn_tool.fn_signature)

{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

TOOL Agent

In [55]:
tool_agent = ToolAgent(tools=[hn_tool],model="llama3-70b-8192")

In [57]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")
print(output)


Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "ESP32 Undocumented Bluetooth Commands: Clearing the Air", "url": "https://developer.espressif.com/blog/2025/03/esp32-bluetooth-clearing-the-air/"}, {"title": "A Supersolid Made Using Photons", "url": "https://www.photonicsonline.com/doc/a-super-solid-made-using-pho-tons-0001"}, {"title": "Show HN: Seven39, a social media app that is only open for 3 hours every evening", "url": "https://www.seven39.com"}, {"title": "Bending Spacetime in the Basement (1997)", "url": "https://www.fourmilab.ch/gravitation/foobar/"}, {"title": "Software-Defined Radio for Engineers (2018) [pdf]", "url": "https://www.analog.com/media/en/training-seminars/design-handbooks/Software-Defined-Radio-for-Engineers-2018/SDR4Engineers.pdf"}]
Here are the top 5 Hacker News stories right now:

1. **ESP32 Undocumented Bluetooth Commands: Clearing the Air** -